In [1]:
import h2o
import numpy as np
import pandas as pd

In [2]:
diamonds =  pd.read_csv('./diamonds-datamad1019/data.csv/data.csv')

In [3]:
color = {
    "D": 7,
    "E": 6,
    "F": 5,
    "G": 4,
    "H": 3,
    "I": 2,
    "J": 1
}

cut = {
    "Ideal": 5,
    "Premium": 4,
    "Very Good": 3,
    "Good": 2,
    "Fair":1
}

clarity = {
    "IF": 8,
    "VVS1": 7,
    "VVS2": 6,
    "VS1": 5,
    "VS2": 4,
    "SI1": 3,
    "SI2": 2,
    "I1": 1
}

In [4]:
for i in range(len(diamonds)):
    co = color[diamonds.loc[i, "color"]]
    cu = cut[diamonds.loc[i, "cut"]]
    cl = clarity[diamonds.loc[i, "clarity"]]
    diamonds.loc[i, "color_num"] = co
    diamonds.loc[i, "cut_num"] = cu
    diamonds.loc[i, "clarity_num"] = cl

In [5]:
diamonds.drop(columns=['cut','color','clarity'], inplace=True)
diamonds['volume'] = diamonds['x']*diamonds['y']*diamonds['z']
diamonds.drop(['x','y','z'],axis=1,inplace=True)

In [6]:
diamonds

,carat,depth,table,price,color_num,cut_num,clarity_num,volume
0,2.26,61.9,57.0,12831,4.0,5.0,2.0,366.903680
1,2.43,63.2,57.0,16170,3.0,3.0,2.0,392.176400
2,0.80,61.0,57.0,2797,5.0,4.0,2.0,133.001901
3,0.40,63.3,60.0,630,5.0,5.0,1.0,64.059840
4,0.31,61.6,55.0,698,4.0,5.0,4.0,51.797610
...,...,...,...,...,...,...,...,...
40450,1.11,62.8,61.0,5315,3.0,4.0,3.0,180.060192
40451,0.73,62.6,56.0,2762,5.0,5.0,4.0,119.231280
40452,1.26,59.2,60.0,6855,2.0,3.0,5.0,208.046124
40453,0.72,61.4,56.0,2297,4.0,5.0,2.0,119.547648


In [8]:
h2o.init(nthreads = -1, max_mem_size = 26)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /home/mariobru/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpyd9bbs7w
  JVM stdout: /tmp/tmpyd9bbs7w/h2o_mariobru_started_from_python.out
  JVM stderr: /tmp/tmpyd9bbs7w/h2o_mariobru_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,00 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_mariobru_4n4h1g
H2O cluster total nodes:,1
H2O cluster free memory:,26 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [9]:
diamonds_h2o=h2o.H2OFrame(diamonds)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
diamonds_h2o.head()

carat,depth,table,price,color_num,cut_num,clarity_num,volume
2.26,61.9,57,12831,4,5,2,366.904
2.43,63.2,57,16170,3,3,2,392.176
0.8,61,57,2797,5,4,2,133.002
0.4,63.3,60,630,5,5,1,64.0598
0.31,61.6,55,698,4,5,4,51.7976
0.53,60.5,60,1132,5,3,2,85.5027
0.7,60.8,61,1987,7,4,2,110.716
1.13,64,58,3952,5,2,2,179.758
0.35,60.9,55,614,6,5,3,57.3448
1.01,61.4,66,5062,3,1,5,153.103


In [84]:
y_columns = "price"
x_columns = ["carat","color_num","cut_num","clarity_num","volume","table"]

In [85]:
train, test=diamonds_h2o.split_frame(ratios = [.8])
X_train=train[x_columns]
y_train=train[y_columns]
X_test=test[x_columns]
y_test=test[y_columns]

In [86]:
from h2o.automl import H2OAutoML
aml_ti = H2OAutoML(max_runtime_secs= 180,max_models= 15, seed= 1, nfolds=0)
aml_ti.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [87]:
lb_ti = aml_ti.leaderboard
lb_ti

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_2_AutoML_20191209_132414,316924,562.96,316924,289.62,0.113998
GBM_1_AutoML_20191209_132414,317080,563.099,317080,287.519,0.11062
GBM_2_AutoML_20191209_132414,319652,565.378,319652,288.084,0.110419
GBM_3_AutoML_20191209_132414,322730,568.093,322730,294.324,nan
XGBoost_3_AutoML_20191209_132414,323564,568.827,323564,293.685,0.118157
GBM_4_AutoML_20191209_132414,328818,573.427,328818,294.063,0.11516
XGBoost_1_AutoML_20191209_132414,328862,573.465,328862,290.429,0.105275
XRT_1_AutoML_20191209_132414,329920,574.386,329920,294.671,0.11055
DRF_1_AutoML_20191209_132414,336281,579.897,336281,292.741,0.110366
GBM_5_AutoML_20191209_132414,336902,580.432,336902,294.81,nan


In [88]:
pred_automl = aml_ti.leader.predict(test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [89]:
pred_automl

predict
618.031
5430.29
6068.63
2891.39
871.352
5190.23
1552.11
5130.24
8252.83
811.189


In [90]:
y_test

price
552
5315
6670
2775
965
5433
1429
4830
8014
707


In [91]:
#Probamos con test

In [92]:
diamonds_test =  pd.read_csv('./diamonds-datamad1019/test.csv')

In [93]:
for i in range(len(diamonds_test)):
    co = color[diamonds_test.loc[i, "color"]]
    cu = cut[diamonds_test.loc[i, "cut"]]
    cl = clarity[diamonds_test.loc[i, "clarity"]]
    diamonds_test.loc[i, "color_num"] = co
    diamonds_test.loc[i, "cut_num"] = cu
    diamonds_test.loc[i, "clarity_num"] = cl

In [95]:
diamonds_test.drop(columns=['cut','color','clarity'], inplace=True)

In [96]:
diamonds_test['volume'] = diamonds_test['x']*diamonds_test['y']*diamonds_test['z']
diamonds_test.drop(['x','y','z'],axis=1,inplace=True)

In [97]:
diamonds_h2o=h2o.H2OFrame(diamonds)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [99]:
x_columns = ["carat","color_num","cut_num","clarity_num","volume","table"]

In [102]:
X_train=diamonds_h2o[x_columns]

In [103]:
pred_automl = aml_ti.leader.predict(X_train)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [104]:
pred_automl

predict
14674.6
16000.3
2610.07
510.234
656.357
1060.46
2151.27
4310.88
670.556
4260.45


In [105]:
submission = pred_automl.as_data_frame()

In [106]:
submission

,predict
0,14674.609375
1,16000.343750
2,2610.066162
3,510.233551
4,656.356995
...,...
40450,4989.118164
40451,2946.767822
40452,6556.066895
40453,2309.455811


In [107]:
submission.rename(columns={"predict": "price"}, inplace=True)
submission

,price
0,14674.609375
1,16000.343750
2,2610.066162
3,510.233551
4,656.356995
...,...
40450,4989.118164
40451,2946.767822
40452,6556.066895
40453,2309.455811


In [108]:
submission.price = submission.price.astype(int)


In [109]:
submission

,price
0,14674
1,16000
2,2610
3,510
4,656
...,...
40450,4989
40451,2946
40452,6556
40453,2309


In [111]:
submission.to_csv('./diamonds-datamad1019/submission.csv')